# Project Proposal

Name : Darwin Alarcon \
Proposal: Analyzing areas of Boston underserved by the MBTA

## Motivation
As the city of Boston continues to grow and expand, the MBTA (Massachusetts Bay Transportation Authority) will need to evolve to meet the increasing transportation demands of the population. Public transit is a critical factor in urban development, and plays a role in economic mobility, reducing traffic congestion, and promoting sustainable living. Despite its importance, there are still areas within Boston where access to reliable public transportation is limited, leading to the creation of "transit deserts." These are neighborhoods where residents face challenges in accessing public transit within a reasonable walking distance, often resulting in longer commutes, reduced mobility, and increased reliance on cars.

Identifying these transit gaps is essential to ensuring equitable access to transportation services across all neighborhoods, particularly for lower-income residents who may depend heavily on public transit. Using Data Science we can systematically analyze public transit availability in Boston and uncover underserved areas. This would give valuable insights to urban planners and policymakers, helping to guide future transit expansions.

So, there are 2 questions we want to answer: \
"Where are the largest gaps in public transportation access when considering population density?" \
"Which neighborhoods should be prioritized for new transit routes or stop improvements based on population density and current distance to public transit?"

Sources:

https://apps.bostonglobe.com/ideas/graphics/2022/06/the-next-big-dig/fixing-t-is-not-enough-we-need-massive-expansion

https://www.youtube.com/watch?v=c-ls4qqfZDA

## Data Sources
- Public Transit Data (Stops and Routes):

MBTA API: The MBTA provides an API that gives detailed information on public transit stops, routes, and real-time service data. We could use this data to map transit accessibility across Boston.

- Population Density:

US Census API: Gives populations for smaller areas, so  good for population density mapping

- Additional:

City of Boston Open Data Portal: The portal provides geo-data on census blocks, which we will tie to our population desnity data to create a folium map.

## Load Data

### MBTA Info and Stops

In [110]:
#libraries
import requests
import pandas as pd
import geopandas as gpd
import folium
import json
import zipfile
import os

In [ ]:
# Load MBTA Data and some sample data from it
api_key = 'Insert API Key' 

# URL to get routes information
url = 'https://api-v3.mbta.com/routes'

# Set the headers including API key
headers = {
    'x-api-key': api_key
}

# Send GET request
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()['data']
    
    routes_df = pd.DataFrame([{
        'short_name': route['attributes'].get('short_name', 'N/A'),
        'long_name': route['attributes'].get('long_name', 'N/A'),
        'description': route['attributes'].get('description', 'N/A'),
        'type': route['attributes'].get('type', 'N/A')
    } for route in data])

    stops_df = pd.DataFrame([{
        'stop_id': stop['id'],
        'stop_name': stop['attributes'].get('name', 'N/A'),
        'latitude': stop['attributes'].get('latitude', 'N/A'),
        'longitude': stop['attributes'].get('longitude', 'N/A')
    } for stop in data])
else:
    print(f"Error: {response.status_code}")

In [112]:
routes_df.head(6)

,short_name,long_name,description,type
0,,Red Line,Rapid Transit,1
1,,Mattapan Trolley,Rapid Transit,0
2,,Orange Line,Rapid Transit,1
3,B,Green Line B,Rapid Transit,0
4,C,Green Line C,Rapid Transit,0
5,D,Green Line D,Rapid Transit,0


In [113]:
#something we also need is the locations of these stops
url = 'https://api-v3.mbta.com/stops'

# Send the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()['data']
    
    # Extract attributes
    stops_df = pd.DataFrame([{
        'stop_id': stop['id'],
        'stop_name': stop['attributes'].get('name', 'N/A'),
        'latitude': stop['attributes'].get('latitude', 'N/A'),
        'longitude': stop['attributes'].get('longitude', 'N/A')
    } for stop in data])
else:
    print(f"Error: {response.status_code}")

In [114]:
stops_df.head(5)

,stop_id,stop_name,latitude,longitude
0,7206,Euclid Ave @ Magnolia Ave,42.487216,-70.955600
1,76592,Moody St @ Underwood Pk,42.361177,-71.239061
2,node-lech-obrienstairs-top,Lechmere,42.370861,-71.075629
3,4601,North St opp Mason St,42.526150,-70.900323
4,3537,309 Bridge St,42.242762,-70.953050


### Population Desnity

In [116]:
# URL for the 2020 Census Tract GeoJSON
census_tract_url = 'https://bostonopendata-boston.opendata.arcgis.com/api/download/v1/items/cc9761b2de8d4b368c6437c5e176caa0/geojson?layers=0'

# Load the GeoJSON data into a GeoDataFrame
census_gdf = gpd.read_file(census_tract_url)

census_gdf.head()

,geoid20,countyfp20,namelsad20,statefp20,tractce20,intptlat20,name20,funcstat20,intptlon20,mtfcc20,aland20,awater20,objectid,geometry
0,25025140202,025,Census Tract,25,140202,+42.2495181,1402.02,S,-071.1175430,G5020,1538599,17120,1,"POLYGON ((-71.12623 42.24268, -71.12624 42.242..."
1,25025140300,025,Census Tract,25,140300,+42.2587734,1403,S,-071.1188131,G5020,1548879,38736,2,"POLYGON ((-71.13012 42.25118, -71.12965 42.250..."
2,25025140400,025,Census Tract,25,140400,+42.2692219,1404,S,-071.1118088,G5020,1874512,11680,3,"POLYGON ((-71.12491 42.27271, -71.1248 42.2725..."
3,25025140106,025,Census Tract,25,140106,+42.2738738,1401.06,S,-071.1371416,G5020,278837,3116,4,"POLYGON ((-71.14069 42.2747, -71.14075 42.2743..."
4,25025110201,025,Census Tract,25,110201,+42.2804960,1102.01,S,-071.1170508,G5020,348208,0,5,"POLYGON ((-71.11999 42.27883, -71.11994 42.278..."


In [ ]:
api_key = 'Insert API Key' 

url = f"https://api.census.gov/data/2020/dec/pl?get=P1_001N,NAME&for=block group:*&in=state:25 county:025&key={api_key}"

response = requests.get(url)

if response.status_code == 200:
    # Convert the API response to a DataFrame
    data = response.json()
    columns = data[0]  # Column names
    population_data = pd.DataFrame(data[1:], columns=columns)

    # Rename the columns for clarity
    population_data.rename(columns={
        'P1_001N': 'population',
        'NAME': 'block_group_name',
        'block group': 'block_group_id'
    }, inplace=True)

    # Convert population to numeric
    population_data['population'] = pd.to_numeric(population_data['population'])

    # Construct the GEOID by combining state, county, tract, and block_group_id so we can tie it to census blocks
    population_data['GEOID'] = (
        population_data['state'] +
        population_data['county'] +
        population_data['tract'] +
        population_data['block_group_id']
    )
else:
    print(f"Error: {response.status_code}")

In [118]:
population_data.head()

,population,block_group_name,state,county,tract,block_group_id,GEOID
0,977,"Block Group 3, Census Tract 1304.02, Suffolk C...",25,025,130402,3,250251304023
1,1034,"Block Group 2, Census Tract 1304.04, Suffolk C...",25,025,130404,2,250251304042
2,569,"Block Group 3, Census Tract 1304.06, Suffolk C...",25,025,130406,3,250251304063
3,1036,"Block Group 1, Census Tract 1403, Suffolk Coun...",25,025,140300,1,250251403001
4,753,"Block Group 4, Census Tract 1403, Suffolk Coun...",25,025,140300,4,250251403004


In [119]:
# URL for the 2020 Census Block Groups GeoJSON
geojson_url = 'https://data.boston.gov/dataset/c478b600-3e3e-46fd-9f57-da89459e9928/resource/98201cf0-8aa9-4751-a34d-4d45191a3456/download/census2020_blockgroups.json'

# Load the GeoJSON data into a GeoDataFrame
block_group_gdf = gpd.read_file(geojson_url)

block_group_gdf.head()

,FID,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,BLKGRPCE20,GEOID20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,Shape_STAr,Shape_STLe,geometry
0,0,1,25,025,040600,1,250250406001,Block Group 1,G5030,S,1265377,413598,+42.3833695,-071.0707743,1.807118e+07,29256.866068,"POLYGON ((769378.692 2964626.314, 769383.713 2..."
1,1,2,25,025,051101,1,250250511011,Block Group 1,G5030,S,220626,0,+42.3882285,-071.0046816,2.374654e+06,9142.174252,"POLYGON ((788317.786 2966115.262, 788438.834 2..."
2,2,3,25,025,051101,4,250250511014,Block Group 4,G5030,S,227071,270,+42.3913407,-071.0020343,2.446949e+06,11579.546171,"POLYGON ((789538.125 2967889.427, 789503.621 2..."
3,3,4,25,025,981600,1,250259816001,Block Group 1,G5030,S,586981,158777,+42.3886205,-070.9934424,8.026752e+06,16626.718823,"POLYGON ((790938.417 2966482.118, 790936.403 2..."
4,4,5,25,025,010204,3,250250102043,Block Group 3,G5030,S,145888,0,+42.3459611,-071.1020344,1.570220e+06,5510.560013,"POLYGON ((762928.668 2951612.031, 762939.909 2..."


In [120]:
population_data['GEOID'] = population_data['GEOID'].astype(str)
block_group_gdf['GEOID'] = block_group_gdf['GEOID20'].astype(str) 

# Merge the population data with the block group GeoDataFrame
merged_gdf = block_group_gdf.merge(population_data, on='GEOID')

# Inspect the merged GeoDataFrame
merged_gdf.head()


,FID,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,BLKGRPCE20,GEOID20,NAMELSAD20,MTFCC20,FUNCSTAT20,...,Shape_STAr,Shape_STLe,geometry,GEOID,population,block_group_name,state,county,tract,block_group_id
0,0,1,25,025,040600,1,250250406001,Block Group 1,G5030,S,...,1.807118e+07,29256.866068,"POLYGON ((769378.692 2964626.314, 769383.713 2...",250250406001,1760,"Block Group 1, Census Tract 406, Suffolk Count...",25,025,040600,1
1,1,2,25,025,051101,1,250250511011,Block Group 1,G5030,S,...,2.374654e+06,9142.174252,"POLYGON ((788317.786 2966115.262, 788438.834 2...",250250511011,1803,"Block Group 1, Census Tract 511.01, Suffolk Co...",25,025,051101,1
2,2,3,25,025,051101,4,250250511014,Block Group 4,G5030,S,...,2.446949e+06,11579.546171,"POLYGON ((789538.125 2967889.427, 789503.621 2...",250250511014,1099,"Block Group 4, Census Tract 511.01, Suffolk Co...",25,025,051101,4
3,3,4,25,025,981600,1,250259816001,Block Group 1,G5030,S,...,8.026752e+06,16626.718823,"POLYGON ((790938.417 2966482.118, 790936.403 2...",250259816001,2,"Block Group 1, Census Tract 9816, Suffolk Coun...",25,025,981600,1
4,4,5,25,025,010204,3,250250102043,Block Group 3,G5030,S,...,1.570220e+06,5510.560013,"POLYGON ((762928.668 2951612.031, 762939.909 2...",250250102043,968,"Block Group 3, Census Tract 102.04, Suffolk Co...",25,025,010204,3


In [122]:
# Calculate area in square miles
merged_gdf['area_sq_miles'] = merged_gdf['Shape_STAr'] / (1609.34 ** 2)  # Convert square meters to square miles

# Calculate population density (people per square mile)
merged_gdf['population_density'] = merged_gdf['population'] / merged_gdf['area_sq_miles']

print(merged_gdf[['GEOID', 'population', 'area_sq_miles', 'population_density']].head())


          GEOID  population  area_sq_miles  population_density
0  250250406001        1760       6.977355          252.244601
1  250250511011        1803       0.916864         1966.486532
2  250250511014        1099       0.944777         1163.237291
3  250259816001           2       3.099162            0.645336
4  250250102043         968       0.606268         1596.652691


In [124]:
# Check the min and max of population density so we can better bin it
print(merged_gdf['population_density'].min(), merged_gdf['population_density'].max())

0.0 19969.945206138946


In [129]:
# Create a Folium map of population density as a proof of concept
boston_map = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

# Define custom bins for population density
bins = [0, 500, 1000, 2000, 3000, 5000, 10000, 20000, 30000]  # Adjust bins based on your data range

# Add a choropleth layer with custom bins
choropleth = folium.Choropleth(
    geo_data=merged_gdf,
    name='choropleth',
    data=merged_gdf,
    columns=['GEOID', 'population_density'],
    key_on='feature.properties.GEOID',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population Density (people per square mile)',
    bins=bins,  # Use custom bins
    reset=True
).add_to(boston_map)

# Add popups for each block group
for _, row in merged_gdf.iterrows():
    folium.GeoJson(
        row['geometry'],
        popup=folium.Popup(f"""
            <strong>GEOID:</strong> {row['GEOID']}<br>
            <strong>Population:</strong> {row['population']}<br>
            <strong>Population Density:</strong> {round(row['population_density'], 2)} per square mile
        """, max_width=300)
    ).add_to(boston_map)

# Save the map to an HTML file
boston_map.save("boston_block_group_population_density_map_with_popups.html")


### Justification

The first question will be addressed by analyzing and mapping the relationship between population density and proximity to MBTA stops. I will use block group population data from the U.S. Census. Using this data, I will calculate the population density for each block group in Boston. Then, I will overlay this population density data with the locations of MBTA stops, retrieved via the MBTA API, to assess transit accessibility. I will calculate the distance from each block group to the nearest MBTA stop. This will allow for a visual analysis of underserved areas, and I will use folium to create a map highlighting these regions. I can also re-run this analysis after completing question 2, to do a before and after.

For the second question, which explores how walkability and distance to MBTA stops vary between neighborhoods, I will calculate the distance from the center of each block group to the nearest MBTA stop. By combining this geographic distance with population density and demographic data, I will identify neighborhoods where walking to transit stops is more challenging, focusing on areas with a higher population density but fewer accessible transit options. This analysis will provide a clearer picture of the challenges residents face in accessing public transportation.

For machine learning, I could use clustering algorithms, such as k-means, which can be used to group neighborhoods with similar transit access challenges based on factors like population density, distance to the nearest transit stop, and current transit availability. Classification models could also be applied to predict which neighborhoods are likely to benefit the most from potential transit expansions, based on their existing transit access and demographics.